In [1]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/yy2895/Desktop/RawData.csv')
d = df.values
ntotal=len(df)


d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(7)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,8):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 14
n_encoder_h_2 = 10



#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 14
n_decoder_h_2 = 19




# Parameters
learning_rate = 0.01

#batch_size = 7
display_step = 1

total_batch=7

training_epochs = 7000
#we store the variables here

def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [19, n_encoder_h_1], [n_encoder_h_1], phase_train)

  
       
        with tf.variable_scope("code"):
            output = layer(h_1, [n_encoder_h_1, n_code], [n_code], phase_train)

    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [n_code, n_decoder_h_1], [n_decoder_h_1], phase_train)

        with tf.variable_scope("output"):
            output = layer(h_1, [n_decoder_h_1, 19], [19], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '10'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 19]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive200=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/yy2895/Desktop/auto19-14-10/tmp/model.ckpt')
                        togive3=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,19)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive3.append(output_any_image)
                            
                        togive200=togive3
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)


we begin
Epoch: 0001 cost = 2.281085048
Validation Loss: 2.7586722
Epoch: 0002 cost = 2.262351240
Validation Loss: 2.3947906
Epoch: 0003 cost = 2.244200536
Validation Loss: 2.2421513
Epoch: 0004 cost = 2.224730628
Validation Loss: 2.1684766
Epoch: 0005 cost = 2.203248705
Validation Loss: 2.0992339
Epoch: 0006 cost = 2.180168799
Validation Loss: 2.0336223
Epoch: 0007 cost = 2.157299076
Validation Loss: 2.0006561
Epoch: 0008 cost = 2.136032139
Validation Loss: 1.8981863
Epoch: 0009 cost = 2.116739648
Validation Loss: 1.8874931
Epoch: 0010 cost = 2.099711214
Validation Loss: 1.841596
Epoch: 0011 cost = 2.084603684
Validation Loss: 1.8432857
Epoch: 0012 cost = 2.070888145
Validation Loss: 1.9768915
Epoch: 0013 cost = 2.057948283
Validation Loss: 2.0007455
Epoch: 0014 cost = 2.045057433
Validation Loss: 2.022224
Epoch: 0015 cost = 2.031261717
Validation Loss: 2.0580378
Epoch: 0016 cost = 2.015697105
Validation Loss: 2.007673
Epoch: 0017 cost = 1.999157378
Validation Loss: 1.9806807
Epoch: 0

Validation Loss: 1.1723455
Epoch: 0145 cost = 0.998680072
Validation Loss: 1.0560752
Epoch: 0146 cost = 0.993776168
Validation Loss: 0.9969089
Epoch: 0147 cost = 0.988903199
Validation Loss: 0.87205553
Epoch: 0148 cost = 0.984060918
Validation Loss: 1.0606706
Epoch: 0149 cost = 0.979249273
Validation Loss: 1.3867905
Epoch: 0150 cost = 0.974468197
Validation Loss: 1.3793583
Epoch: 0151 cost = 0.969717153
Validation Loss: 1.0579741
Epoch: 0152 cost = 0.964996278
Validation Loss: 1.1201618
Epoch: 0153 cost = 0.960305146
Validation Loss: 0.8950118
Epoch: 0154 cost = 0.955643858
Validation Loss: 0.9396785
Epoch: 0155 cost = 0.951011973
Validation Loss: 1.1248999
Epoch: 0156 cost = 0.946409481
Validation Loss: 1.224731
Epoch: 0157 cost = 0.941836127
Validation Loss: 1.2961608
Epoch: 0158 cost = 0.937291682
Validation Loss: 1.2283852
Epoch: 0159 cost = 0.932776119
Validation Loss: 1.1800209
Epoch: 0160 cost = 0.928289167
Validation Loss: 0.95781386
Epoch: 0161 cost = 0.923830697
Validation Lo

Epoch: 0291 cost = 0.529026500
Validation Loss: 0.36553815
Epoch: 0292 cost = 0.527050001
Validation Loss: 0.39267153
Epoch: 0293 cost = 0.525085134
Validation Loss: 0.4841095
Epoch: 0294 cost = 0.523131890
Validation Loss: 0.5309882
Epoch: 0295 cost = 0.521190235
Validation Loss: 0.53994805
Epoch: 0296 cost = 0.519260049
Validation Loss: 0.6669937
Epoch: 0297 cost = 0.517341209
Validation Loss: 0.7191675
Epoch: 0298 cost = 0.515433665
Validation Loss: 0.68720084
Epoch: 0299 cost = 0.513537377
Validation Loss: 0.48741
Epoch: 0300 cost = 0.511652270
Validation Loss: 0.45522735
Epoch: 0301 cost = 0.509778223
Validation Loss: 0.72105324
Epoch: 0302 cost = 0.507915143
Validation Loss: 0.56050044
Epoch: 0303 cost = 0.506062976
Validation Loss: 0.43156984
Epoch: 0304 cost = 0.504221699
Validation Loss: 0.3304353
Epoch: 0305 cost = 0.502391143
Validation Loss: 0.29453346
Epoch: 0306 cost = 0.500571349
Validation Loss: 0.4485585
Epoch: 0307 cost = 0.498762148
Validation Loss: 0.38585895
Epoch:

Validation Loss: 0.39852792
Epoch: 0437 cost = 0.332809550
Validation Loss: 0.34783962
Epoch: 0438 cost = 0.331930522
Validation Loss: 0.28144687
Epoch: 0439 cost = 0.331154619
Validation Loss: 0.39053896
Epoch: 0440 cost = 0.330552331
Validation Loss: 0.497968
Epoch: 0441 cost = 0.330083932
Validation Loss: 0.39072728
Epoch: 0442 cost = 0.329469327
Validation Loss: 0.3677994
Epoch: 0443 cost = 0.328357211
Validation Loss: 0.37165233
Epoch: 0444 cost = 0.327083064
Validation Loss: 0.3057281
Epoch: 0445 cost = 0.326356994
Validation Loss: 0.3042552
Epoch: 0446 cost = 0.325941984
Validation Loss: 0.33160016
Epoch: 0447 cost = 0.325071292
Validation Loss: 0.36993483
Epoch: 0448 cost = 0.323851270
Validation Loss: 0.3976924
Epoch: 0449 cost = 0.323116392
Validation Loss: 0.43262917
Epoch: 0450 cost = 0.322536026
Validation Loss: 0.35673055
Epoch: 0451 cost = 0.321351877
Validation Loss: 0.43210062
Epoch: 0452 cost = 0.320231838
Validation Loss: 0.3839761
Epoch: 0453 cost = 0.319445091
Vali

Epoch: 0578 cost = 0.245895569
Validation Loss: 0.2178087
Epoch: 0579 cost = 0.245399956
Validation Loss: 0.22247246
Epoch: 0580 cost = 0.244913576
Validation Loss: 0.22324778
Epoch: 0581 cost = 0.244514291
Validation Loss: 0.20532435
Epoch: 0582 cost = 0.244150932
Validation Loss: 0.21387137
Epoch: 0583 cost = 0.243780347
Validation Loss: 0.21198888
Epoch: 0584 cost = 0.243411449
Validation Loss: 0.1942615
Epoch: 0585 cost = 0.243030429
Validation Loss: 0.20035644
Epoch: 0586 cost = 0.242600558
Validation Loss: 0.2062672
Epoch: 0587 cost = 0.242148433
Validation Loss: 0.23565106
Epoch: 0588 cost = 0.241736921
Validation Loss: 0.24925376
Epoch: 0589 cost = 0.241371438
Validation Loss: 0.24323438
Epoch: 0590 cost = 0.241017635
Validation Loss: 0.21905345
Epoch: 0591 cost = 0.240661298
Validation Loss: 0.19696039
Epoch: 0592 cost = 0.240304821
Validation Loss: 0.19258547
Epoch: 0593 cost = 0.239935151
Validation Loss: 0.19173722
Epoch: 0594 cost = 0.239534797
Validation Loss: 0.23007426


...

Validation Loss: 0.024395622
Epoch: 6403 cost = 0.026382901
Validation Loss: 0.02460634
Epoch: 6404 cost = 0.026381824
Validation Loss: 0.02430885
Epoch: 6405 cost = 0.026380683
Validation Loss: 0.024524568
Epoch: 6406 cost = 0.026379636
Validation Loss: 0.024764704
Epoch: 6407 cost = 0.026378536
Validation Loss: 0.025150677
Epoch: 6408 cost = 0.026377437
Validation Loss: 0.027134523
Epoch: 6409 cost = 0.026376349
Validation Loss: 0.026289877
Epoch: 6410 cost = 0.026375258
Validation Loss: 0.026257409
Epoch: 6411 cost = 0.026374174
Validation Loss: 0.025223816
Epoch: 6412 cost = 0.026373130
Validation Loss: 0.024840921
Epoch: 6413 cost = 0.026372032
Validation Loss: 0.024940154
Epoch: 6414 cost = 0.026370971
Validation Loss: 0.026659576
Epoch: 6415 cost = 0.026369922
Validation Loss: 0.025865668
Epoch: 6416 cost = 0.026368841
Validation Loss: 0.025366906
Epoch: 6417 cost = 0.026367787
Validation Loss: 0.027878076
Epoch: 6418 cost = 0.026366743
Validation Loss: 0.029241562
Epoch: 6419 c

Validation Loss: 0.02401566
Epoch: 6548 cost = 0.026248553
Validation Loss: 0.024864731
Epoch: 6549 cost = 0.026247750
Validation Loss: 0.026030755
Epoch: 6550 cost = 0.026246981
Validation Loss: 0.025737979
Epoch: 6551 cost = 0.026246217
Validation Loss: 0.026187072
Epoch: 6552 cost = 0.026245419
Validation Loss: 0.025056414
Epoch: 6553 cost = 0.026244624
Validation Loss: 0.024685392
Epoch: 6554 cost = 0.026243870
Validation Loss: 0.025159888
Epoch: 6555 cost = 0.026243079
Validation Loss: 0.025204059
Epoch: 6556 cost = 0.026242328
Validation Loss: 0.02482832
Epoch: 6557 cost = 0.026241521
Validation Loss: 0.024682077
Epoch: 6558 cost = 0.026240755
Validation Loss: 0.02462426
Epoch: 6559 cost = 0.026240020
Validation Loss: 0.02393918
Epoch: 6560 cost = 0.026239235
Validation Loss: 0.02528841
Epoch: 6561 cost = 0.026238444
Validation Loss: 0.02566914
Epoch: 6562 cost = 0.026237697
Validation Loss: 0.02442267
Epoch: 6563 cost = 0.026236942
Validation Loss: 0.02430172
Epoch: 6564 cost = 

Validation Loss: 0.027343601
Epoch: 6685 cost = 0.026153258
Validation Loss: 0.026887866
Epoch: 6686 cost = 0.026152648
Validation Loss: 0.026438719
Epoch: 6687 cost = 0.026152043
Validation Loss: 0.025335172
Epoch: 6688 cost = 0.026151445
Validation Loss: 0.025245959
Epoch: 6689 cost = 0.026150824
Validation Loss: 0.023806876
Epoch: 6690 cost = 0.026150220
Validation Loss: 0.024352215
Epoch: 6691 cost = 0.026149626
Validation Loss: 0.025018733
Epoch: 6692 cost = 0.026148999
Validation Loss: 0.024446784
Epoch: 6693 cost = 0.026148418
Validation Loss: 0.02350494
Epoch: 6694 cost = 0.026147785
Validation Loss: 0.042472992
Epoch: 6695 cost = 0.026147205
Validation Loss: 0.02788789
Epoch: 6696 cost = 0.026146557
Validation Loss: 0.024644077
Epoch: 6697 cost = 0.026145930
Validation Loss: 0.024510337
Epoch: 6698 cost = 0.026145343
Validation Loss: 0.025975816
Epoch: 6699 cost = 0.026144756
Validation Loss: 0.025168018
Epoch: 6700 cost = 0.026144147
Validation Loss: 0.024786066
Epoch: 6701 c

Validation Loss: 0.023471026
Epoch: 6828 cost = 0.026070643
Validation Loss: 0.023963617
Epoch: 6829 cost = 0.026070038
Validation Loss: 0.023869634
Epoch: 6830 cost = 0.026069480
Validation Loss: 0.023721391
Epoch: 6831 cost = 0.026068925
Validation Loss: 0.024214037
Epoch: 6832 cost = 0.026068336
Validation Loss: 0.024806308
Epoch: 6833 cost = 0.026067805
Validation Loss: 0.024604743
Epoch: 6834 cost = 0.026067255
Validation Loss: 0.024326324
Epoch: 6835 cost = 0.026066724
Validation Loss: 0.023635477
Epoch: 6836 cost = 0.026066228
Validation Loss: 0.023878295
Epoch: 6837 cost = 0.026065712
Validation Loss: 0.024028845
Epoch: 6838 cost = 0.026065239
Validation Loss: 0.025480706
Epoch: 6839 cost = 0.026064761
Validation Loss: 0.02473201
Epoch: 6840 cost = 0.026064284
Validation Loss: 0.025093067
Epoch: 6841 cost = 0.026063856
Validation Loss: 0.024964932
Epoch: 6842 cost = 0.026063400
Validation Loss: 0.02413626
Epoch: 6843 cost = 0.026062985
Validation Loss: 0.023660325
Epoch: 6844 c

Epoch: 6965 cost = 0.026003901
Validation Loss: 0.024172295
Epoch: 6966 cost = 0.026003132
Validation Loss: 0.024568066
Epoch: 6967 cost = 0.026002363
Validation Loss: 0.024143567
Epoch: 6968 cost = 0.026001647
Validation Loss: 0.023578878
Epoch: 6969 cost = 0.026000949
Validation Loss: 0.024399951
Epoch: 6970 cost = 0.026000270
Validation Loss: 0.028000241
Epoch: 6971 cost = 0.025999617
Validation Loss: 0.029302236
Epoch: 6972 cost = 0.025998977
Validation Loss: 0.029106915
Epoch: 6973 cost = 0.025998385
Validation Loss: 0.0248126
Epoch: 6974 cost = 0.025997785
Validation Loss: 0.023811946
Epoch: 6975 cost = 0.025997231
Validation Loss: 0.023443855
Epoch: 6976 cost = 0.025996736
Validation Loss: 0.023654949
Epoch: 6977 cost = 0.025996272
Validation Loss: 0.024281519
Epoch: 6978 cost = 0.025995922
Validation Loss: 0.024245597
Epoch: 6979 cost = 0.025995640
Validation Loss: 0.02559151
Epoch: 6980 cost = 0.025995434
Validation Loss: 0.025994169
Epoch: 6981 cost = 0.025995327
Validation L

In [2]:
currentmin

0.023261553

In [3]:
import csv
with open('C:/Users/yy2895/Desktop/auto_result19-14-10.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive200)):
        wr.writerow(list(togive200[i][0]))

In [4]:
togive200

[array([[0.3686394 , 0.87191993, 0.656978  , 0.42828804, 0.55214083,
         0.37255085, 0.12607822, 0.38795033, 0.56586415, 0.5870667 ]],
       dtype=float32),
 array([[0.3819131 , 0.87590146, 0.64456856, 0.5068038 , 0.54082525,
         0.36509213, 0.11899576, 0.41472486, 0.58213514, 0.5147113 ]],
       dtype=float32),
 array([[0.41939044, 0.8374763 , 0.5809937 , 0.6014107 , 0.5113255 ,
         0.37798446, 0.11195317, 0.4533126 , 0.6314493 , 0.4727664 ]],
       dtype=float32),
 array([[0.5070742 , 0.851583  , 0.5042515 , 0.75846255, 0.4554945 ,
         0.40494254, 0.06786037, 0.4825412 , 0.659291  , 0.4854649 ]],
       dtype=float32),
 array([[0.5785083 , 0.8276119 , 0.39765158, 0.86027795, 0.40156358,
         0.46484342, 0.0506671 , 0.5185871 , 0.69031596, 0.46566227]],
       dtype=float32),
 array([[0.5190523 , 0.844805  , 0.4597731 , 0.7231944 , 0.49441874,
         0.41053453, 0.08922201, 0.42754865, 0.66411513, 0.4942133 ]],
       dtype=float32),
 array([[0.59899724, 0